In [2]:
import numpy as np
from numpy.lib.scimath import sqrt as csqrt, log as clog
import os

# Given DCP model functions from the provided code snippet
def epsilon_cp1(omega, omega_g1, omega01, gamma1, A1):
    i = 1j
    term1 = omega + i * gamma1

    # Fraction 1: Arctan term
    sqrt_arg1 = (omega_g1 - omega01) / (term1 - omega_g1)
    arctan_term = np.arctan(csqrt(sqrt_arg1))
    fraction1 = -csqrt(term1 - omega_g1) * arctan_term / (term1**2)

    # Fraction 2: Atanh term
    sqrt_arg2 = (omega_g1 - omega01) / (term1 + omega_g1)
    atanh_term = np.arctanh(csqrt(sqrt_arg2))
    fraction2 = -csqrt(term1 + omega_g1) * atanh_term / (term1**2)

    # Fraction 3: Atanh term for omega_g1
    atanh_term_g1 = np.arctanh(csqrt((omega_g1 - omega01) / omega_g1))
    fraction3 = 2 * csqrt(omega_g1) * atanh_term_g1 / (term1**2)

    # Log term
    log_arg = 1 - ((term1 / omega01)**2)
    log_term = np.log(log_arg)
    fraction4 = -csqrt(omega_g1 - omega01) * log_term / (2 * term1**2)

    # Combine all terms
    epsilon = A1 * (fraction1 + fraction2 + fraction3 + fraction4)
    return epsilon

def epsilon_cp2(omega, omega02, gamma2, A2):
    i = 1j
    term2 = (omega + (i * gamma2))

    fraction = A2 / (2 * (term2**2))
    log_arg = 1 - ((term2 / omega02)**2)
    log_term = clog(log_arg)

    epsilon = -fraction * log_term
    return epsilon

def epsilon_total(omega, epsilon_inf, omega_p, gamma_p, epsilon_cp1_val, epsilon_cp2_val):
    i = 1j
    term_p = omega**2 + (i * gamma_p * omega)
    plasma_term = (omega_p**2) / term_p
    epsilon = epsilon_inf - plasma_term + epsilon_cp1_val + epsilon_cp2_val
    return epsilon

def calculate_nk(epsilon):
    epsilon_real = np.real(epsilon)
    epsilon_imag = np.imag(epsilon)

    epsilon_magnitude = np.sqrt(epsilon_real**2 + epsilon_imag**2)
    n = np.sqrt((epsilon_magnitude + epsilon_real) / 2)
    k = np.sqrt((epsilon_magnitude - epsilon_real) / 2)
    return n, k


# Parameters from Table 2
# Format: xAg, epsilon_inf, omega_p, gamma_p, A1, omega01, omega1, gamma1, A2, omega02, omega2, gamma2
params_table = [
 (0.00, 2.568, 8.905, 0.109, 56.39, 2.342, 2.714, 0.125, 32.13, 3.516, 3.516, 0.264),
 (0.08, 2.621, 9.115, 0.108, 56.50, 2.363, 2.770, 0.149, 33.02, 3.564, 3.564, 0.257),
 (0.15, 2.522, 9.062, 0.135, 52.67, 2.428, 2.820, 0.166, 33.78, 3.568, 3.568, 0.292),
 (0.24, 2.429, 8.986, 0.174, 51.38, 2.525, 2.866, 0.195, 33.68, 3.583, 3.583, 0.289),
 (0.35, 2.398, 9.056, 0.175, 46.16, 2.580, 2.951, 0.190, 34.49, 3.626, 3.626, 0.289),
 (0.48, 2.193, 8.970, 0.196, 41.42, 2.735, 3.079, 0.189, 35.33, 3.717, 3.717, 0.291),
 (0.66, 2.047, 8.820, 0.179, 23.73, 2.689, 3.372, 0.164, 35.98, 3.833, 3.833, 0.292),
 (0.74, 2.027, 8.974, 0.159, 29.09, 2.839, 3.770, 0.134, 31.19, 4.090, 4.090, 0.293),
 (0.85, 2.022, 9.116, 0.126, 33.42, 3.086, 3.997, 0.109, 25.15, 4.224, 4.224, 0.237),
 (1.00, 1.909, 9.104, 0.086, 62.13, 3.759, 4.155, 0.045, 18.29, 4.796, 4.796, 0.217)
]

# Create output directory
out_dir = "dielectric_data"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Define wavelength range (in micrometers)
# Adjust as desired. Example: 0.18 to 2.0 µm, 100 points
wavelengths = np.linspace(0.18, 2.0, 200)

for entry in params_table:
    (xAg, epsilon_inf, omega_p, gamma_p, A1, omega01, omega1, gamma1, A2, omega02, omega2, gamma2) = entry

    # Compute Au fraction
    xAu = 1.0 - xAg


    # Round percentages
    Au_percent = round(xAu*100)
    Ag_percent = round(xAg*100)
    filename = f"Pena-Rioux_dielectric_{Au_percent}Au_{Ag_percent}Ag"
    filepath = os.path.join(out_dir, filename)

    with open(filepath, 'w') as f:
        # Write headers
        f.write(f"Peña-Rodríguez & Rioux calculated dielectric data ({Au_percent}Au/{Ag_percent}Ag) prepared by Alexander Al-Feghali al-fegha.li, Blum and Cosa Lab, McGill University\n")
        f.write("1 2 3 0 0 = columns for wave, Re(n), Im(n), eps1, eps2\n")
        f.write("wave(um)   Re(n)     Im(n)     eps1      eps2\n")

        for w in wavelengths:
            # Convert wavelength (um) to energy (eV): E(eV) = 1.23984193 / wavelength(um)
            omega = 1.23984193 / w

            # Calculate epsilon_cp1 and epsilon_cp2
            epsilon_cp1_val = epsilon_cp1(omega, omega1, omega01, gamma1, A1)
            epsilon_cp2_val = epsilon_cp2(omega, omega02, gamma2, A2)

            # Calculate total epsilon
            eps = epsilon_total(omega, epsilon_inf, omega_p, gamma_p, epsilon_cp1_val, epsilon_cp2_val)

            # Extract eps1 and eps2
            eps1 = np.real(eps)
            eps2 = np.imag(eps)

            # Calculate n and k
            n, k = calculate_nk(eps)

            # Write data line
            f.write(f"{w:0.6f}\t{n:0.6f}\t{k:0.6f}\t{eps1:0.6f}\t{eps2:0.6f}\n")
